#### Helpful links
####  https://github.com/Reagan0914/DInSAR_Processing_Snappy/blob/master/DataProcessing/S1_SLC_Processing.py
#### https://www.youtube.com/watch?v=PiU68g3WRIY&t=1498s


### Import libraries


In [ ]:
import os
import subprocess
import shutil
import snappy_esa
from snappy_esa import ProductIO
from snappy_esa import GPF
from snappy_esa import HashMap
from snappy_esa import jpy
from datetime import datetime
import sys
from multiprocessing import Process
import pandas as pd
import csv
from os.path import join 
from glob import iglob
from concurrent.futures import ProcessPoolExecutor
from datetime import datetime

## Definition
Introduction:
 This script contains functions to perform various operations on Sentinel-1 SAR (Synthetic Aperture Radar) products.
 These operations include data manipulation, processing, and correction to prepare the data for further analysis.

### Description:
 The script defines functions to perform the following operations:
 - Reading and writing Sentinel-1 SAR products.
 - Splitting TOPSAR data into subswaths.
 - Assembling slices of TOPSAR data.
 - Combining multiple products.
 - Applying orbit files for precise orbit information.
 - Back-geocoding to correct geometric distortions.
 - Calculating enhanced spectral diversity.
 - Generating interferograms.
 - Debursting TOPSAR data.
 - Performing terrain correction to project data onto a common coordinate reference system.
 Each function takes input parameters, applies processing steps using the SNAP (Sentinel Application Platform) toolbox, and returns the processed product.

 Usage:
 Users can call these functions individually or combine them to create custom processing pipelines for Sentinel-1 SAR data.
 For example, they can read data, perform terrain correction, and then generate interferograms for interferometric analysis.


In [ ]:
HashMap = jpy.get_type('java.util.HashMap')

parameters = HashMap()

def read(filename):
    return ProductIO.readProduct(filename)

def write(product, filename, format=None):
    return ProductIO.writeProduct(product, filename, format if format else "GeoTIFF")

def TOPSAR_split1(product, firstBurstIndex, lastBurstIndex):
    parameters = HashMap()
    parameters.put('subswath', 'IW1')
    parameters.put('selectedPolarisations', 'VV')
    parameters.put('firstBurstIndex', firstBurstIndex)
    parameters.put('lastBurstIndex', lastBurstIndex)
    return GPF.createProduct('TOPSAR-Split', parameters, product)

def TOPSAR_split2(product, firstBurstIndex, lastBurstIndex):
    parameters = HashMap()
    parameters.put('subswath', 'IW2')
    parameters.put('selectedPolarisations', 'VV')
    parameters.put('firstBurstIndex', firstBurstIndex)
    parameters.put('lastBurstIndex', lastBurstIndex)
    return GPF.createProduct('TOPSAR-Split', parameters, product)

def TOPSAR_split3(product, firstBurstIndex, lastBurstIndex):
    parameters = HashMap()
    parameters.put('subswath', 'IW3')
    parameters.put('selectedPolarisations', 'VV')
    parameters.put('firstBurstIndex', firstBurstIndex)
    parameters.put('lastBurstIndex', lastBurstIndex)
    return GPF.createProduct('TOPSAR-Split', parameters, product)

def Slice_Assembly(list_of_products):
    parameters = HashMap()
    parameters.put('selectedPolarizations', 'VV')
    return GPF.createProduct('SliceAssembly', parameters, list_of_products)

def combine():
    file1 = read(os.path.join(path_files_to_be_merged, files[0]))
    file2 = read(os.path.join(path_files_to_be_merged, files[1]))
    file3 = read(os.path.join(path_files_to_be_merged, files[2]))
    merged_product = Slice_Assembly([TOPSAR_split1(file1, 1, 9), TOPSAR_split2(file2, 1, 9), TOPSAR_split3(file3, 1, 9)])
    write(merged_product, os.path.join(output_dir, f"S1_SLC_{files[0].split('_')[5][:8]}_{files[0].split('_')[-1][:-4]}_{files[2].split('_')[-1][:-4]}_Split_SplAmb"))
    return merged_product

def apply_orbit_file(product):
    parameters = HashMap()
    parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
    parameters.put('polyDegree', 3)
    parameters.put('continueOnFail', True)
    return GPF.createProduct('Apply-Orbit-File', parameters, product)

def back_geocoding(product):    
    parameters = HashMap()
    parameters.put("Digital Elevation Model", "SRTM 1Sec HGT")
    parameters.put("DEM Resampling Method", "BICUBIC_INTERPOLATION")
    parameters.put("Resampling Type", "BISINC_5_POINT_INTERPOLATION")
    parameters.put("Mask out areas with no elevation", True)
    parameters.put("Output Deramp and Demod Phase", True)    
    return GPF.createProduct("Back-Geocoding", parameters, product)

def enhanced_spectral_diversity(product):
    parameters = HashMap()
    parameters.put("fineWinWidthStr", "512")  
    parameters.put("fineWinHeightStr", "512") 
    parameters.put("fineWinAccAzimuth", "16")  
    parameters.put("fineWinAccRange", "16")  
    parameters.put("fineWinOversampling", "128")  
    parameters.put("xCorrThreshold", "0.1")  
    parameters.put("cohThreshold", "0.15")  
    parameters.put("numBlocksPerOverlap", "10")  
    parameters.put("useSuppliedRangeShift", False)
    parameters.put("overallRangeShift", "0.0")  
    parameters.put("useSuppliedAzimuthShift", False)
    parameters.put("overallAzimuthShift", "0.0")  
    return GPF.createProduct('Enhanced-Spectral-Diversity', parameters, product)

def interferogram(product):
    parameters = HashMap()  
    parameters.put("Subtract flat-earth phase", True)
    parameters.put("Degree of \"Flat Earth\" polynomial", 5)
    parameters.put("Number of \"Flat Earth\" estimation points", 501)
    parameters.put("Orbit interpolation degree", 3)
    parameters.put("Include coherence estimation", True)
    parameters.put("Square Pixel", True)
    parameters.put("Independent Window Sizes", False)
    parameters.put("Coherence Azimuth Window Size", 5)
    parameters.put("Coherence Range Window Size", 20)
    return GPF.createProduct("Interferogram", parameters, product)

def topsar_deburst(product):
    parameters = HashMap()
    parameters.put('selectedPolarisations','VV')
    return GPF.createProduct('TOPSAR-Deburst', parameters, product)

def terrain_correction(src, projection):
    parameters = HashMap()
    parameters.put("demName", "SRTM 1Sec HGT")  # ~25 to 30m
    parameters.put("externalDEMNoDataValue", 0.0)
    parameters.put("externalDEMApplyEGM", True)
    parameters.put("demResamplingMethod", "BICUBIC_INTERPOLATION")
    parameters.put("imgResamplingMethod", "BICUBIC_INTERPOLATION")
    parameters.put("pixelSpacingInMeter", 10.0)
    parameters.put("pixelSpacingInDegree", 8.983152841195215E-5)
    parameters.put("mapProjection", projection)
    parameters.put("alignToStandardGrid", False)
    parameters.put("standardGridOriginX", 0.0)
    parameters.put("standardGridOriginY", 0.0)
    parameters.put("nodataValueAtSea", True)
    parameters.put("saveDEM", False)
    parameters.put("saveLatLon", False)
    parameters.put("saveIncidenceAngleFromEllipsoid", False)
    parameters.put("saveLocalIncidenceAngle", False)
    parameters.put("saveSelectedSourceBand", True)
    parameters.put("outputComplex", False)
    parameters.put("applyRadiometricNormalization", False)
    parameters.put("saveSigmaNought", False)
    parameters.put("saveGammaNought", False)
    parameters.put("saveBetaNought", False)
    parameters.put("incidenceAngleForSigma0", "Use projected local incidence angle from DEM")
    parameters.put("incidenceAngleForGamma0", "Use projected local incidence angle from DEM")
    parameters.put("auxFile", "Latest Auxiliary File")
    return GPF.createProduct("Terrain-Correction", parameters, src)


## Data for calculation
### Images along the path
 Description:
 This script processes Sentinel-1 SAR data for interferometric analysis. It performs the following steps:

 1. Define paths to Sentinel-1 SLC (Single Look Complex) data files.
 2. Read the data from the specified files.
 3. Split the TOPSAR data of both master and slave images into subswaths (burst 1 to 9).
 4. Apply orbit files to correct for precise orbit information.
 5. Perform back-geocoding to correct geometric distortions induced by satellite motion.
 6. Calculate enhanced spectral diversity to improve coherence estimation.
 7. Generate interferograms by computing the phase difference between master and slave images.
 8. Deburst the TOPSAR data to remove burst synchronization artifacts.
 9. Perform terrain correction to project the data onto a common coordinate reference system (CRS).
 10. Save the terrain-corrected product in BEAM-DIMAP format.

 Usage:
 Users need to specify the paths to the input Sentinel-1 SLC data files (filename_1 and filename_2).
 The script then processes these files sequentially to perform the mentioned steps.
 Finally, it saves the terrain-corrected product to the specified output filename.

 Note:
 This script assumes the availability of SNAP (Sentinel Application Platform) toolbox functions for SAR data processing.
 Users should have SNAP installed and configured to use the provided functions like TOPSAR split, back-geocoding, etc.
 Additionally, users should adjust the paths and filenames according to their specific data locations and requirements.


In [ ]:
filename_1 = ''
filename_2 = ''
read1 = read(filename_1)
read2 = read(filename_2)
master_TOPSAR_split = TOPSAR_split1(read1, 1, 9)
slave_TOPSAR_split = TOPSAR_split1(read2, 1, 9)
master_orbitFile = apply_orbit_file(master_TOPSAR_split)
slave_orbitFile = apply_orbit_file(slave_TOPSAR_split)
backGeocoding = back_geocoding([slave_orbitFile, master_orbitFile])
enhanced_spectral_diversity_product = enhanced_spectral_diversity(backGeocoding)
interferogram_product = interferogram(enhanced_spectral_diversity_product)
TOPSAR_deburst_product = topsar_deburst(interferogram_product)
terrain_correction_product = terrain_correction(TOPSAR_deburst_product, "EPSG:32631")
filename = ""
write(terrain_correction_product, filename, "BEAM-DIMAP")


## Merging and Processing Sentinel-1 SAR Data
This Python script performs data merging and processing tasks on Sentinel-1 Synthetic Aperture Radar (SAR) data obtained from the Alaska Satellite Facility (ASF). Below is a breakdown of the script's functionalities:
### Merging CSV Files:
The script reads two CSV files containing metadata obtained from the ASF data pool. These CSV files are merged together to consolidate the metadata information.
### Generating File Paths:
A function `change_name()` is defined to convert the granule names into corresponding file paths. It extracts the date information from the granule names and constructs the file paths accordingly.
### Extracting Date Information:
The script extracts the date from the granule names and converts it into a datetime format for sorting and further processing.
### Sorting Data:
The data is sorted based on the extracted date information to ensure chronological order.
### Saving Processed Data:
The processed data, consisting of file paths and dates, is saved to a new CSV file.
This script streamlines the initial data processing steps required for further analysis of Sentinel-1 SAR data.

In [ ]:
#merging two csv from Alaska Facility 
df1 = pd.read_csv('')
df2 = pd.read_csv('')
data = pd.concat([df1, df2], ignore_index=True)
column = data['Granule Name']

def change_name(name):
    date = name.split('_')[5]
    year = date[:4]
    month = date[4:6]
    day = date[6:8]
    new_name = f'/{year}/{month}/{day}/{name}.SAFE'
    return new_name
df = pd.DataFrame({'Granule Name': column, 'Path': column.apply(change_name)})
df['Date'] = df['Granule Name'].str.extract(r'_(\d{8})T')[0]
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d', errors='coerce')
df = df.sort_values(by='Date')
df.to_csv('', columns=['Path', 'Date'], index=False, header=False)

In [ ]:
with open('', 'r') as file:
    csv_reader = csv.reader(file)
    rows = list(csv_reader)

for i in range(0, len(rows) - 1):
    path1, date1 = rows[i] 
    path2, date2 = rows[i + 1] 

    date1 = datetime.strptime(date1, "%Y-%m-%d").strftime("%Y%m%d")
    date2 = datetime.strptime(date2, "%Y-%m-%d").strftime("%Y%m%d")

    filename_1 = path1
    filename_2 = path2
    read1 = read(filename_1)
    read2 = read(filename_2)

    master_TOPSAR_split = TOPSAR_split1(read1, 1, 9)
    slave_TOPSAR_split = TOPSAR_split1(read2, 1, 9)
    print("TOPSAR split")

    master_orbitFile = apply_orbit_file(master_TOPSAR_split)
    slave_orbitFile = apply_orbit_file(slave_TOPSAR_split)
    print("Apply orbit file")

    backGeocoding = back_geocoding([slave_orbitFile, master_orbitFile])
    print("Back geocoding")

    enhanced_spectral_diversity_product = enhanced_spectral_diversity(backGeocoding)
    print("Enhanced spectral diversity")

    interferogram_product = interferogram(enhanced_spectral_diversity_product)
    print("Interferogram")

    TOPSAR_deburst_product = topsar_deburst(interferogram_product)
    print("TOPSAR deburst")

    terrain_correction_product = terrain_correction(TOPSAR_deburst_product, "EPSG:32631")
    print("Terrain correction")

    output_filename = "" + date1 + "_" + date2
    write(terrain_correction_product, output_filename, "BEAM-DIMAP")
    print("Write")
    

    read1.dispose()
    read2.dispose()
    master_TOPSAR_split.dispose()
    slave_TOPSAR_split.dispose()
    master_orbitFile.dispose()
    slave_orbitFile.dispose()
    backGeocoding.dispose()
    enhanced_spectral_diversity_product.dispose()
    interferogram_product.dispose()
    TOPSAR_deburst_product.dispose()
    terrain_correction_product.dispose()
    gc.collect()